In [123]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import sklearn as skl
import folium
from folium.plugins import HeatMap
from statsmodels.formula.api import ols

%matplotlib inline

In [124]:
df = pd.read_csv("data/kc_house_data.csv")

df['sqft_basement'] = np.where(df['sqft_basement']=='?', 0, df['sqft_basement'])
df['sqft_basement'] = df['sqft_basement'].astype(float)
df['condition'] = np.where(df['condition']=='?', 0, df['condition'])
df['condition'] = df['condition'].astype(float)
df = df.fillna(0)
df = df.drop(columns=['id'])
df['has_basement1'] = np.where(df['sqft_basement']=='?', 0, df['sqft_basement'])
df['has_basement1'] = df['has_basement1'].astype(float)
df['has_basement']  = np.where(df['has_basement1']>0, 1, 0)
df = df.sort_values(['bedrooms'], ascending=False)[1:]
df = df.drop(columns=['has_basement1'])
df['sqft_first_floor'] = df['sqft_above']/df['floors']
df['devland'] = df['sqft_first_floor']/df['sqft_lot']
df['devland_outdated'] = df['sqft_above']/df['sqft_lot']
df['sqft_comp_neighbors'] = df['sqft_living']/df['sqft_living15']
df['is_renovated'] = np.where(df['yr_renovated']>0, 1, df['yr_renovated'])
df['age'] = 2015 - df['yr_built']

In [125]:
listy  = []
listy1 = []
listy2 = []
listy3 = []
listy4 = []
listy5 = []

for each in df.index:
    if df.loc[each]['bathrooms'] <= 1.5:
        listy.append(1)
    else:
        listy.append(0)

for each in df.index:
    if df.loc[each]['grade'] <= 7:
        listy1.append(1)
    else:
        listy1.append(0)

for each in df.index:
    if df.loc[each]['devland'] <= .23:
        listy2.append(1)
    else:
        listy2.append(0)

for each in df.index:
    if df.loc[each]['has_basement'] == 1:
        listy3.append(0.5)
    else:
        listy3.append(0)

for each in df.index:
    if df.loc[each]['is_renovated'] == 1:
        listy4.append(0.5)
    else:
        listy4.append(0)

for each in df.index:
    if df.loc[each]['age'] >= 50:
        listy5.append(-1)
    else:
        listy5.append(0)

In [126]:
listy00 = pd.DataFrame(listy)
listy01 = pd.DataFrame(listy1)
listy02 = pd.DataFrame(listy2)
listy03 = pd.DataFrame(listy3)
listy04 = pd.DataFrame(listy4)
listy05 = pd.DataFrame(listy5)

df['intrinsic_value'] = listy00 + listy01 + listy02 +listy03 + listy04 + listy05

In [127]:
dfscore = df[['price', 'bathrooms', 'grade', 'devland', 'has_basement', 'is_renovated', 'age']]

In [128]:
# fig, ax = plt.subplots(figsize=(30,30))
# sns.heatmap(df.corr(), cmap='coolwarm', annot=True)
# pd.plotting.scatter_matrix(df, figsize=(10,10))

# fig, ax = plt.subplots(figsize=(15,15))
# sns.heatmap(df_log.corr(), cmap='coolwarm', annot=True)

In [129]:
dfy = pd.DataFrame([])
dfy['price_log']         = np.log(df['price'])
dfy['bedrooms_log']      = np.log(df['bedrooms'])
dfy['bathrooms_log']     = np.log(df['bathrooms'])
dfy['sqft_lot_log']      = np.log(df['sqft_lot'])
dfy['floors_log']        = np.log(df['floors'])
dfy['sqft_above_log']    = np.log(df['sqft_above'])
dfy['yr_built_log']      = np.log(df['yr_built'])
dfy['sqft_living15_log'] = np.log(df['sqft_living15'])
dfy['sqft_lot15_log']    = np.log(df['sqft_lot15'])
dfy['devland_log']       = np.log(df['devland'])

df['price_log']          = np.log(df['price'])
df['bedrooms_log']       = np.log(df['bedrooms'])
df['bathrooms_log']      = np.log(df['bathrooms'])
df['sqft_lot_log']       = np.log(df['sqft_lot'])
df['floors_log']         = np.log(df['floors'])
df['sqft_above_log']     = np.log(df['sqft_above'])
df['yr_built_log']       = np.log(df['yr_built'])
df['sqft_living15_log']  = np.log(df['sqft_living15'])
df['sqft_lot15_log']     = np.log(df['sqft_lot15'])
df['devland_old_log']    = np.log(df['devland_outdated'])

In [130]:
dfx = df[['price','bedrooms_log','bathrooms','sqft_living','sqft_lot_log','waterfront','view','condition','grade','yr_built','devland_old_log']]

In [131]:
df.head()

,date,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,condition,...,price_log,bedrooms_log,bathrooms_log,sqft_lot_log,floors_log,sqft_above_log,yr_built_log,sqft_living15_log,sqft_lot15_log,devland_old_log
8748,8/21/2014,520000.0,11,3.00,3000,4960,2.0,0.0,0.0,3.0,...,13.161584,2.397895,1.098612,8.509161,0.693147,7.783224,7.559038,7.258412,8.509161,-0.725937
13301,8/14/2014,1150000.0,10,5.25,4590,10920,1.0,0.0,2.0,3.0,...,13.955273,2.302585,1.658228,9.298351,0.000000,7.824046,7.604894,7.912057,9.249561,-1.474305
19239,12/29/2014,660000.0,10,3.00,2920,3745,2.0,0.0,0.0,4.0,...,13.399995,2.302585,1.098612,8.228177,0.693147,7.528332,7.556428,7.501082,8.228177,-0.699845
15147,10/29/2014,650000.0,10,2.00,3610,11914,2.0,0.0,0.0,4.0,...,13.384728,2.302585,0.693147,9.385469,0.693147,8.009695,7.579679,7.620705,9.385469,-1.375774
6073,8/8/2014,1280000.0,9,4.50,3650,5000,2.0,0.0,0.0,3.0,...,14.062371,2.197225,1.504077,8.517193,0.693147,7.835975,7.557473,7.828038,8.517193,-0.681219


In [132]:
formula = "price ~ bedrooms_log+bathrooms+sqft_living+sqft_lot_log+waterfront+view+condition+grade+yr_built+devland_old_log"
testy = ols(formula = formula, data = dfx).fit()
testy.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                  price   R-squared:                       0.661
Model:                            OLS   Adj. R-squared:                  0.661
Method:                 Least Squares   F-statistic:                     4208.
Date:                Wed, 17 Feb 2021   Prob (F-statistic):               0.00
Time:                        17:54:39   Log-Likelihood:            -2.9570e+05
No. Observations:               21596   AIC:                         5.914e+05
Df Residuals:                   21585   BIC:                         5.915e+05
Df Model:                          10                                         
Covariance Type:            nonrobust                                         
===================================================================================
                      coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------
Intercept        6.499e+06   1.27e+05     51.214      0.000    6.25e+06    6.75e+06
bedrooms_log    -1.311e+05   6648.451    -19.712      0.000   -1.44e+05   -1.18e+05
bathrooms        4.079e+04   3359.135     12.142      0.000    3.42e+04    4.74e+04
sqft_living       209.7490      3.898     53.813      0.000     202.109     217.389
sqft_lot_log    -1.069e+05   7132.553    -14.987      0.000   -1.21e+05   -9.29e+04
waterfront       6.245e+05   1.92e+04     32.444      0.000    5.87e+05    6.62e+05
view             4.185e+04   2195.755     19.061      0.000    3.75e+04    4.62e+04
condition        1.856e+04   2429.368      7.641      0.000    1.38e+04    2.33e+04
grade            1.312e+05   2141.119     61.279      0.000    1.27e+05    1.35e+05
yr_built        -3330.7208     66.239    -50.283      0.000   -3460.554   -3200.887
devland_old_log -7.305e+04   7158.205    -10.205      0.000   -8.71e+04    -5.9e+04
==============================================================================
Omnibus:                    15363.352   Durbin-Watson:                   1.983
Prob(Omnibus):                  0.000   Jarque-Bera (JB):           925061.596
Skew:                           2.812   Prob(JB):                         0.00
Kurtosis:                      34.566   Cond. No.                     2.57e+05
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 2.57e+05. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

In [133]:
dfg = df.groupby('zipcode')
dfg

In [134]:
Mappy = folium.Map(location = [47.6062,-122.3321], zoom_start = 9)

